In [3]:
# numpy and pandas for data manipulation
import sys
import os
# Suppress warnings
import warnings

import matplotlib.pyplot as plt
import networkx as nx

import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
import cPickle as pickle

warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline

ROOT = '/Users/bede01/Documents/work/phd-papers/ml-workload-optimization/'
ROOT_PACKAGE_DIRECTORY = '/Users/bede01/Documents/work/phd-papers/ml-workload-optimization/code/collaborative-optimizer'
root_data = ROOT + '/data'

sys.path.append(ROOT_PACKAGE_DIRECTORY)
from experiment_graph.executor import CollaborativeExecutor
from experiment_graph.workload import Workload
from experiment_graph.execution_environment import ExecutionEnvironment
from experiment_graph.benchmark_helper import BenchmarkMetrics
from experiment_graph.data_storage import DedupedStorageManager
from experiment_graph.storage_managers import storage_profiler

DATABASE_PATH = root_data + '/experiment_graphs/kaggle_home_credit/start_here_a_gentle_introduction/all'
N_ESTIMATOR = 100

In [4]:
# Prepare the Data
# Create Execution Environment with AllMaterializer
local_profile = storage_profiler.get_profile(root_data+'/profiles/local-dedup')
ee = ExecutionEnvironment(DedupedStorageManager()) 
ee.load_history_from_disk(DATABASE_PATH)
#workload = MaterializationTest()
executor = CollaborativeExecutor(ee, cost_profile=local_profile)
#executor.end_to_end_run(workload=workload, root_data=root_data)

{"Agg": 0.09642666666666667, "SK_Model": 5.86940990518854e-05, "Evaluation": 0.04334545454545455, "Feature": 0.03777813765182186, "Dataset": 0.0006677262236078122}


In [5]:
executor.compute_heuristics(ee.experiment_graph.graph, executor.DEFAULT_PROFILE)

In [4]:
graph = executor.execution_environment.experiment_graph.graph

In [8]:
0.1171875 * 0.08959999999999999

0.010499999999999999

In [6]:
max_load = (0,0)
for n,d in graph.nodes(data=True):
    if 'load_cost' in d:
        print n, d['type'], d['size'], d['load_cost'], d['recreation_cost']
        if max_load[1] < d['load_cost']:
            max_load = (n,d['load_cost'])

91F663F204EC1B373BAA7D5F4ED259B3 Feature 1.9296875 4.9599200026e-05 53975020.108
ECDA0D538C2716511D31B918A0A196AB Agg 0.1171875 0.0105 6751138.855
6D67B47B385303BCE343069471903013 Feature 1921.9375 0.0494 4683.694
421BD661A585C601515158BB06A45BCE Feature 1921.9375 0.0494 36.653
2FDDD27929272F0872ED90A8782B3E9E Feature 1.8984375 4.87959741147e-05 341815.233
6AABC0FEF0CC9CEE313FEE2F88BCD237 Dataset 29309.6171875 14.7703 247.386
EDD0B28901B73F1F09CD86A338953753 Dataset 116759.671875 58.8399149147 265146.411
7DC7FD1982D2E6DFAC4689879CF74496 Feature 480.4921875 0.0123502008065 489.107
588E87EA4494D973E7E1D3023591BA38 Feature 13327.8173828 0.342567944437 0.159
1EBBC23A724A33FC623DFFE910804FFA Agg 0.1171875 0.0105 336292.717
5EDB6467754F13B3F2AAA2F921144194 Dataset 235437.414062 118.646422936 4850.955
B049A25988BB3F5CCB75BF7894073109 Agg 0.0234375 0.0021 4671.389
A1AC5FEB094CCC27E0FBE2C877097260 Feature 240.2421875 0.006175 82971.496
167A31E89EBD1BC62655FD7052DA2ABE Feature 1.921875 4.9398393

E37F048B899D318CD4BA87C7C9A44DDA Feature 11574.4785156 0.297501473733 4658.48
3F632739DF0CD1B6FBB40EE6E4992F10 Feature 1.9296875 4.9599200026e-05 26987509.639
31253C26E2FF11A3F9CBFC52C49D8E91 Agg 0.03125 0.0028 165948.338
03B5B1A6B7E6A529298BE6F96046608E Dataset 5291.34375 2.66652184812 248942.122
9E9BAF3CED5C7D225138BDFE337CD0C8 Dataset 244566.617188 123.246997146 6749764.148
2CDD5AAC44505D882479B8DB63ED5148 Dataset 3843.9453125 1.93711931091 82973.041
7B72196374C75F99BCAFE4009CEBCB64 Feature 480.4921875 0.0123502008065 12552.749
77977327B1E4C5F63B993B07FA74C52A Agg 0.0234375 0.0021 4671.816
666D049B5A66C9625117B3BB7CF3C0B5 Feature 1921.9375 0.0494 1498310.284
AE25B96EA42A278A305BCAEAA0DFC5A5 Dataset 42855.1259766 21.5964290206 20421.355
91CAB33ADB2480351D2AEDC0AB8F420A Feature 240.3984375 0.00617901612956 165979.624
354DE3595708334A81873F03DDA5E905 Feature 1921.9375 0.0494 82970.967
65BED3C9DAC84FCBBC03B2DFA8E39FCC Feature 13821.6757812 0.355261700026 4658.485
4733C453477DADC1096DD8B

In [ ]:
graph.nodes['B81B4D2E69928650629AD1266B67FD53']['data']

In [ ]:
graph.nodes['B81B4D2E69928650629AD1266B67FD53']

In [ ]:
for source, destionation, data in graph.edges(data=True):
    if destionation == 'B81B4D2E69928650629AD1266B67FD53':
        print destionation, data

In [ ]:
graph.nodes['application_train.csv{}']['data'].underlying_data.pandas_df

In [ ]:
obj = graph.nodes['E4211A925EA65EB27C7AB0E2EB202872']['data']

In [ ]:
ee.experiment_graph.get_total_size()

In [ ]:
ee.experiment_graph.get_real_size()

In [ ]:
from pympler.asizeof import asizeof

In [ ]:
total = 0
columns = []
for n,d in graph.nodes(data=True):
    if d['type'] == 'Feature':
        print d['data'].underlying_data.pandas_series

            

In [ ]:
(ee.experiment_graph.get_total_size())/(1024 * 1024)

In [ ]:
def utf8len(s):
    return len(s.encode('utf-8'))

In [ ]:
str_size

In [ ]:
np.where(optimized['AMT_ANNUITY'] != baseline['AMT_ANNUITY'])

In [ ]:
#pd.concat([optimized['AMT_ANNUITY'],baseline['AMT_ANNUITY']],axis=1)
optimized['AMT_ANNUITY'][688]

In [ ]:
baseline['AMT_ANNUITY'][688]

In [ ]:
(optimized['AMT_ANNUITY']==baseline['AMT_ANNUITY']).all()

In [ ]:
print optimized['ENTRANCES_MODE'] == baseline['ENTRANCES_MODE']

In [ ]:
baseline['ENTRANCES_MODE'].head()

In [ ]:
start = datetime.now()
ee.new_workload()
# Load Data
train = ee.load(root_data + '/openml/task_id=31/train.csv')
test = ee.load(root_data + '/openml/task_id=31/test.csv')

test_labels = test['class']
test = test.drop('class')

train_labels = train['class']
train = train.drop(columns=['class'])

train2=train.drop('checking_status')
test2=test.drop('checking_status')
from experiment_graph.sklearn_helper.preprocessing import MinMaxScaler
scaler2 = MinMaxScaler(feature_range=(0, 1))
scaler2.fit(train2)
train2 = scaler2.transform(train2)
test2 = scaler2.transform(test2)


scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

# # Random Forest 2 random_state = 60 (n_estimator = 10)
from experiment_graph.sklearn_helper.ensemble import RandomForestClassifier
random_forest2 = RandomForestClassifier(should_warmstart=True, n_estimators=10000, random_state=50, verbose = 1)
random_forest2.fit(train, train_labels)
random_forest2.score(test, test_labels).data()
print 'total time: {}'.format((datetime.now() - start).total_seconds())

In [ ]:
start = datetime.now()
ee.new_workload()
# Load Data
train = ee.load(root_data + '/openml/task_id=31/train.csv')
test = ee.load(root_data + '/openml/task_id=31/test.csv')

test_labels = test['class']
test = test.drop('class')

train_labels = train['class']
train = train.drop(columns=['class'])

train2=train.drop('checking_status')
test2=test.drop('checking_status')
from experiment_graph.sklearn_helper.preprocessing import MinMaxScaler
scaler2 = MinMaxScaler(feature_range=(0, 1))
scaler2.fit(train2)
train2 = scaler2.transform(train2)
test2 = scaler2.transform(test2)


scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

# # Random Forest 2 random_state = 60 (n_estimator = 10)
from experiment_graph.sklearn_helper.ensemble import RandomForestClassifier
random_forest2 = RandomForestClassifier(should_warmstart=True, n_estimators=10000, random_state=60, verbose = 1)
random_forest2.fit(train, train_labels)

# Execute
#random_forest2.trained_node.data()

random_forest2.score(test, test_labels).data()
print 'total time: {}'.format((datetime.now() - start).total_seconds())

In [ ]:
start = datetime.now()
ee.new_workload()
# Load Data
train = ee.load(root_data + '/openml/task_id=31/train.csv')
test = ee.load(root_data + '/openml/task_id=31/test.csv')

test_labels = test['class']
test = test.drop('class')

train_labels = train['class']
train = train.drop(columns=['class'])

train2=train.drop('checking_status')
test2=test.drop('checking_status')
from experiment_graph.sklearn_helper.preprocessing import MinMaxScaler
scaler2 = MinMaxScaler(feature_range=(0, 1))
scaler2.fit(train2)
train2 = scaler2.transform(train2)
test2 = scaler2.transform(test2)


scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

# # Random Forest 2 random_state = 60 (n_estimator = 10)
from experiment_graph.sklearn_helper.ensemble import RandomForestClassifier
random_forest2 = RandomForestClassifier(should_warmstart=False, n_estimators=10000, random_state=60, verbose = 1)
random_forest2.fit(train, train_labels)

# Execute
#random_forest2.trained_node.data()

random_forest2.score(test, test_labels).data()
print 'total time: {}'.format((datetime.now() - start).total_seconds())